# Setup

### API Key

In [167]:
api_key = ""

### Install Packages

In [168]:
!pip install langchain
!pip install langchain_groq
!pip install numexpr
!pip install wikipedia

# LangChain Demo

### Calling LLMs in LangChain

One of the features of langchain is that it helps create an easy to use interface on top of many different LLMs.

In [185]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=1, groq_api_key=api_key, model_name="llama3-70b-8192")

response = llm.predict("Hello")

response

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

### Prompts

In [170]:
from langchain.prompts import ChatPromptTemplate

convert_code_template_string = """Convert the code that is delimited by triple backticks \
from {input_lang} to {output_lang}. \
code: ```{code}```    
"""

convert_code_prompt_template = ChatPromptTemplate.from_template(convert_code_template_string)

print("Prompt Template Object:", convert_code_prompt_template.messages[0].prompt)

Prompt Template Object: input_variables=['code', 'input_lang', 'output_lang'] template='Convert the code that is delimited by tripple backticks from {input_lang} to {output_lang}. code: ```{code}```    \n'


In [171]:
input_lang = "JavaScript"
output_lang = "Python"
code = """
class Node {
  constructor(value) {
    this.value = value;
    this.next = null;
  }
}

class LinkedList {
  constructor() {
    this.head = null;
    this.tail = null;
    this.length = 0;
  }

  append(value) {
    const newNode = new Node(value);

    if (!this.head) {
      this.head = newNode;
      this.tail = newNode;
    } else {
      this.tail.next = newNode;
      this.tail = newNode;
    }

    this.length++;
  }
}
"""

In [172]:
prompt = convert_code_prompt_template.format_messages(input_lang=input_lang, output_lang=output_lang, code=code)
prompt

[HumanMessage(content='Convert the code that is delimited by tripple backticks from JavaScript to Python. code: ```\nclass Node {\n  constructor(value) {\n    this.value = value;\n    this.next = null;\n  }\n}\n\nclass LinkedList {\n  constructor() {\n    this.head = null;\n    this.tail = null;\n    this.length = 0;\n  }\n\n  append(value) {\n    const newNode = new Node(value);\n\n    if (!this.head) {\n      this.head = newNode;\n      this.tail = newNode;\n    } else {\n      this.tail.next = newNode;\n      this.tail = newNode;\n    }\n\n    this.length++;\n  }\n}\n```    \n')]

In [186]:
response = llm.invoke(prompt).content
print(response)

Here is the equivalent code in Python:
```
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, value):
        new_node = Node(value)

        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

        self.length += 1
```
Note that in Python, we use `__init__` instead of `constructor`, and we don't need to use `this` to refer to the current object. Additionally, we use `None` instead of `null` to represent the absence of a value.


### Chains

In [174]:
from langchain.chains import SequentialChain
from langchain.chains import LLMChain

In [189]:
convert_code_chain = LLMChain(llm=llm, prompt=convert_code_prompt_template, output_key="converted_code", verbose=False)
response = convert_code_chain.run({"input_lang": input_lang, "output_lang": output_lang, "code": code})
print(response)

Here is the equivalent code in Python:
```
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, value):
        new_node = Node(value)

        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

        self.length += 1
```
Note that in Python, we use `__init__` method to initialize objects, and `self` to refer to the current object. I've also removed the `const` keyword, as it's not needed in Python.


In [176]:
test_code_template_string = """Write unit tests for the code that is delimited by triple backticks. ```{converted_code}```"""

test_code_prompt_template = ChatPromptTemplate.from_template(test_code_template_string)
print("Prompt Template Object:", test_code_prompt_template.messages[0].prompt)

Prompt Template Object: input_variables=['converted_code'] template='Write unit tests for the code that is delimited by tripple backticks. ```{converted_code}```'


In [177]:
test_code_chain = LLMChain(llm=llm, prompt=test_code_prompt_template, output_key="test_code")

In [192]:
convert_test_code_chain = SequentialChain(
    chains=[convert_code_chain, test_code_chain],
    input_variables=["input_lang", "output_lang", "code"],
    output_variables=["converted_code", "test_code"]
)
result = convert_test_code_chain.invoke({"input_lang": input_lang, "output_lang": output_lang, "code": code})

print("Converted Code:", result["converted_code"])
print("Test Code", result["test_code"])

Converted Code: Here is the equivalent code in Python:
```
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, value):
        new_node = Node(value)

        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

        self.length += 1
```
Note that in Python, we don't need to use `this` to refer to instance variables, and we use `self` instead. Additionally, we use `__init__` to define the constructor, and we don't need to specify the types of variables.
Test Code Here are some unit tests for the provided code:
```
import unittest

class TestNode(unittest.TestCase):
    def test_node_init(self):
        node = Node(1)
        self.assertEqual(node.value, 1)
        self.assertIsNone(node.ne

### Agents / Tools

In [179]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [181]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

agent("What year was the oldest person to ever live born? How many years ago were they born from the current year 2024?")



> Entering new AgentExecutor chain...
Question: What year was the oldest person to ever live born? How many years ago were they born from the current year 2024?

Thought: I need to find the birth year of the oldest person to ever live.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Jeanne Calment"
}
```

Observation: Page: Jeanne Calment
Summary: Jeanne Louise Calment (French: [ʒan lwiz kalmɑ̃] ; 21 February 1875 – 4 August 1997) was a French supercentenarian and, with a documented lifespan of 122 years and 164 days, is the world's oldest verified person. Her longevity attracted media attention and medical studies of her health and lifestyle. She is the only person verified to have reached the age of 120.   
According to census records, Calment outlived both her daughter and grandson. In January 1988, she was widely reported to be the oldest living person, and in 1995, at age 120, was declared the oldest person to have ever lived.

Page: Oldest people
Summary: This is a l

{'input': 'What year was the oldest person to ever live born? How many years ago were they born from the current year 2024?',
 'output': 'Jeanne Calment, the oldest person to ever live, was born in 1875, and as of 2024, she was born 149 years ago.'}

In [182]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathematics should occur \
    outside this function."""
    return str(date.today())

In [184]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm) + [time]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, # LLM might output something that can't be parsed. Pass it back to the LLM and have it correct when this is true.
    verbose=True
)

agent("What year was the oldest person to ever live born? How many years ago were they born from the current year?")



> Entering new AgentExecutor chain...
Question: What year was the oldest person to ever live born? How many years ago were they born from the current year?

Thought: I need to find out who the oldest person to ever live is and when they were born. I can use Wikipedia to find this information.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Oldest people"
}
```

Observation: Page: Oldest people
Summary: This is a list of tables of the oldest people in the world in ordinal ranks. To avoid including false or unconfirmed claims of old age, names here are restricted to those people whose ages have been validated by an international body dealing in longevity research, such as the Gerontology Research Group or Guinness World Records, and others who have otherwise been reliably sourced.
The longest documented and verified human lifespan is that of Jeanne Calment of France, a woman who lived to age 122 years and 164 days. As females live longer than males on average, women predomin

{'input': 'What year was the oldest person to ever live born? How many years ago were they born from the current year?',
 'output': 'Jeanne Calment was born in 1875, which is 149 years ago.'}